- Please run this after running "word2vec-16dim.ipynb"

In [1]:
from sklearn.cluster import SpectralClustering
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import cudf, itertools
import cuml, cupy
from cuml.neighbors import NearestNeighbors
import muon
import scanpy as sc
print('We will use RAPIDS version',cudf.__version__)

cudf.set_option("default_integer_bitwidth", 32)
cudf.set_option("default_float_bitwidth", 32)

We will use RAPIDS version 22.10.00a+392.g1558403753


In [3]:
w2v_path = '../../input/preprocess/'
output_path = '../../input/preprocess/'

In [4]:
w2v = pd.read_parquet(w2v_path + 'test_' + 'w2v_output_16dims.parquet')
w2v = w2v.sort_values('aid').reset_index(drop=True)

In [ ]:
%%time
X_all = sc.AnnData(X = w2v.iloc[:,1:].values)
sc.pp.neighbors(X_all, use_rep = 'X', n_neighbors=64, method='umap')
sc.tl.leiden(X_all)

In [ ]:
aid_df = w2v[['aid']].copy()
aid_df['cluster'] = list(X_all.obs['leiden'])
aid_df['cluster'] = aid_df['cluster'].astype(np.int16)
aid_df.to_parquet(output_path + 'aid_cluster.parquet')